What are we going to do?\
We need to load the chosen/best models for all objects.\
We need to load the test samples for the objects.
we need to run all samples under each model and obtain the error estimation.\
This can be done with different time predictions (starting with 3/2/1 last seconds)\
After we have all error estimations - classify each sample\
evaluate accuracy etc. vs labels.

###Imports

In [8]:
!pip install "pandas<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 50.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [1]:
# imports

import io
import glob
import os
import sys
from datetime import datetime
from time import time
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
# from scipy.fft import fft
import pandas as pd
import seaborn as sns
import random
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr

# Machine Learning
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score

# torch
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential
from torch.nn import Sigmoid,ReLU
from torch.nn import Linear,Dropout
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor,Compose
from torch.optim import SparseAdam,Adam,Adagrad,SGD

!pip install einops
# from __future__ import annotations
from dataclasses import dataclass
from einops import rearrange, repeat, einsum

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# prompt: Access a folder on google drive and import the data locally
from google.colab import drive
drive.mount('/content/drive/')
# helper files
sys.path.append('/content/drive/MyDrive/Final Project UAV/')
from UAV_project_preprocessing_and_visualization_helper_functions_full import *
from mamba_model import *

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Final Project UAV/

/content/drive/MyDrive/Final Project UAV


In [3]:
subfolders = os.listdir("track_data/")
subf_dict = {i:subfolders[i] for i in range(len(subfolders))}
labels_dict = {subfolders[i]:i for i in range(len(subfolders))}

###preparing test samples

In [ ]:
# def prepare_test(subfolder, subfolder_data, batch_size, shuffle = False):
#   subfolder_batches = []
#   subfolder_labels = []
#   for samples, samples_filenames, dt, scales in subfolder_data:
#   # for samples, samples_filenames, dt, scales in all_data[-2:]: #[-2] temp to save time
#     # print(len(samples))

#     if len(samples) > 0:
#       data = list(zip(samples, dt, scales))
#       dl = DataLoader(data, batch_size = batch_size, shuffle = shuffle)
#       print(len(dl))
#       for batch in dl:
#         subfolder_batches.append(batch)
#         print(len(subfolder_batches))
#         subfolder_labels.extend([labels_dict[subfolder]] * len(batch))
#   return subfolder_batches, subfolder_labels

In [ ]:
# def prepare_test_data(subfolder, subfolder_data, batch_size, shuffle = False):
#   subfolder_batches = []
#   subfolder_labels = []
#   for dur, (samples, samples_filenames, dt, scales) in subfolder_data.items():
#     dur_vec = [dur]*len(samples)
#     if len(samples) > 0:
#       data = list(zip(samples, dt, scales, dur_vec))
#       dl = DataLoader(data, batch_size = batch_size, shuffle = shuffle)
#       for batch in dl:
#         subfolder_batches.append(batch)
#         subfolder_labels.extend([labels_dict[subfolder]] * len(batch))
#   return subfolder_batches, subfolder_labels

In [ ]:
# Here we combine the test samples from all different objects (subfolders)
# Each sample duration is combined separately so that we can evaluate separately for each duration
# The model returns the error averaged over the predictions of the last 3 sec of the sequence
# For example - a 5 sec sample would have 2 sec of input sequence without evaluation, and the error will be averaged over the last 3.
# For example - a 30 sec sample would have 27 sec of input sequence without evaluation, and the error will be averaged over the last 3.

# subfolders = ['uav', 'bird']
# durations = [5, 10, 30]
# test_batch_size = 32
# all_test_batches = []
# all_test_labels = []
# for subfolder in subfolders:
#   # dur_batches = {d:[] for d in durations}
#   # dur_labels = {d:[] for d in durations}
#   save_path = './Samples/mamba_samples_testing0_' + subfolder + '_all_samples'
#   with open(save_path , 'rb') as f:
#     # train_samples, test_samples, train_samples_filenames, test_samples_filenames, train_dt, test_dt, train_scales, test_scales = pickle.load(f)
#     (subfolder_train_data, subfolder_vel_data, subfolder_test_data) = pickle.load(f)
#   # subfolder_batches = prepare_test_data(subfolder_test_data, test_batch_size)
#   subfolder_batches, subfolder_labels = prepare_test_data(subfolder, subfolder_test_data, test_batch_size)

#   # subfolder_labels.extend([labels_dict[subfolder]] * len(samples))
#   all_test_batches.extend(subfolder_batches)
#   all_test_labels.extend(subfolder_labels)

In [4]:
# Here we combine the test samples from all different objects (subfolders)
# Each sample duration is combined separately so that we can evaluate separately for each duration
# The model returns the error averaged over the predictions of the last 3 sec of the sequence
# For example - a 5 sec sample would have 2 sec of input sequence without evaluation, and the error will be averaged over the last 3.
# For example - a 30 sec sample would have 27 sec of input sequence without evaluation, and the error will be averaged over the last 3.

subfolders = ['airplane', 'uav', 'bird', 'static-object']
skips = [20, 15, 2, 40]
durations = [5, 10, 15, 20, 25, 30]
test_batch_size = 16
# all_test_batches = {}
# all_test_labels = {}
all_test_batches = {d:[] for d in durations}
all_test_labels = {d:[] for d in durations}
for subfolder, skip in zip(subfolders, skips):
  # dur_batches = {d:[] for d in durations}
  # dur_labels = {d:[] for d in durations}
  save_path = './Samples/mamba_samples_' + subfolder +'_skip'+ str(skip) + '_all_samples'
  with open(save_path , 'rb') as f:
    # train_samples, test_samples, train_samples_filenames, test_samples_filenames, train_dt, test_dt, train_scales, test_scales = pickle.load(f)
    (subfolder_train_data, subfolder_vel_data, subfolder_test_data) = pickle.load(f)
    # subfolder_batches, subfolder_labels = prepare_test(subfolder, subfolder_test_data, test_batch_size)

    # for dur in durations:
    #   subfolder_batches = []
    #   subfolder_labels = []

    for dur, (samples, samples_filenames, dt, scales) in subfolder_test_data.items():
      if len(samples) > 0:
        subfolder_batches = []
        subfolder_labels = []
        dur_vec = [dur]*len(samples)
        data = list(zip(samples, dt, scales, dur_vec))

      # samples, samples_filenames, dt, scales = subfolder_test_data[dur]
      # # for samples, samples_filenames, dt, scales in all_data[-2:]: #[-2] temp to save time
      #   # print(len(samples))

      # if len(samples) > 0:
      #   data = list(zip(samples, dt, scales))
        dl = DataLoader(data, batch_size = test_batch_size, shuffle = False)
        for batch in dl:
          subfolder_batches.append(batch)

        # print(len(dl))
        # print(len(subfolder_batches))
        # Note that the batches are not full so the labels are added as the original number of samples and not based on the number of batches
        subfolder_labels.extend([labels_dict[subfolder]] * len(samples))
        all_test_batches[dur].extend(subfolder_batches)
        all_test_labels[dur].extend(subfolder_labels)

In [ ]:
# subfolders = ['uav', 'bird']
# test_batch_size = 32
# all_test_batches = []
# all_test_labels = []
# for subfolder in subfolders:
#   save_path = './Samples/mamba_samples_testing0_' + subfolder + '_all_samples'
#   with open(save_path , 'rb') as f:
#     # train_samples, test_samples, train_samples_filenames, test_samples_filenames, train_dt, test_dt, train_scales, test_scales = pickle.load(f)
#     (subfolder_train_data, subfolder_vel_data, subfolder_test_data) = pickle.load(f)
#     subfolder_batches, subfolder_labels = prepare_test(subfolder, subfolder_test_data, test_batch_size)
#     sample, dt, scale = subfolder_batches[0]
#     print(sample.shape)
#     print(dt.shape)
#     print(scale.shape)
#     all_test_batches.extend(subfolder_batches)
#     all_test_labels.extend(subfolder_labels)

###Evaluation class

In [5]:
class Evaluating(object):

  def __init__(self, model, config):
    self.config = config
    self.device = config['device']
    self.model = model.to(self.device)
    # self.model_path = config['model_path']
    self.criterion = nn.MSELoss(reduction='none').to(self.device)
    self.evaluation_length = int(config['evaluation_time']/config['delta'])
    self.evaluation_durations = config['evaluation_durations']
    # self.criterion = config['criterion'](weight = config['class_weights']).to(self.device)

  def evaluate(self, dl_eval):
    self.eval_results = {d:[] for d in self.evaluation_durations}
    # self.eval_results = {}
    # self.eval_results[60] = []
    # self.eval_results[30] = []
    # self.eval_results[10] = []
    # self.eval_results[5] = []
    self.epoch_eval_loss = 0

    self.model.eval() #evaluation mode
    with torch.no_grad():
      for batch in tqdm(dl_eval, disable=True):
        self.eval_batch(batch)

  def eval_batch(self, batch):
    data, dt, scale, dur = batch
    # Send tensors to GPU
    samples = data[:,:-1,:]
    labels = data[:,1:,:]
    samples = samples.to(self.device)
    labels = labels.to(self.device)
    dt = dt[:, :-1].to(self.device)
    scale = scale.to(self.device)

    predictions = self.model(samples, dt)#.type(torch.float64)#.type(torch.DoubleTensor)

    scaled_labels = einsum(labels, 1/scale, 'b l d, b d -> b l d')
    # print(scaled_labels)
    scaled_predictions = einsum(predictions, 1/scale, 'b l d, b d -> b l d')
    # print(scaled_predictions)
    eval_loss = self.criterion(scaled_predictions, scaled_labels)
    self.eval_results[int(dur[0])].extend(list((eval_loss[:, -self.evaluation_length:, :].mean((1,2))).cpu()))

    self.epoch_eval_loss += eval_loss.mean().item()

### loading models, running and receiving error

In [4]:
field = 'Best Result @10'

In [5]:
subfolders = ['airplane', 'uav', 'bird', 'static-object']
all_results_df = pd.DataFrame()
for subfolder in subfolders:
  prefix = './Mamba Training Results/mamba_training_results_p_' + subfolder
  folder_results_df = pd.DataFrame()
  for file in glob.glob(prefix + '*'):
    print(file)
    with open(file, 'rb') as f:
      folder_results_df = folder_results_df.append(pickle.load(f))
      # print(pickle.load(f))
  folder_results_df['Object'] = subfolder
  # best_model_row = folder_results_df.loc[folder_results_df['Best Result @5'].idxmin()]
  # all_results_df = all_results_df.append(best_model_row)
  all_results_df = all_results_df.append(folder_results_df)
  all_results_df = all_results_df.drop_duplicates(field)
best_models = all_results_df.groupby('Object').apply(lambda x: x[x[field] == x[field].min()])
# best_models = all_results_df.loc[best_idx].reset_index()
saved_models_path = best_models[['Object', 'Model Path']].set_index('Object').to_dict()['Model Path']
# saved_models_path = all_results_df[['Object', 'Model Path']].set_index('Object').to_dict()['Model Path']

./Mamba Training Results/mamba_training_results_p_airplane04_01_2024, 17:30:00
./Mamba Training Results/mamba_training_results_p_airplane04_01_2024, 17:45:15
./Mamba Training Results/mamba_training_results_p_uav04_01_2024, 08:38:45
./Mamba Training Results/mamba_training_results_p_uav04_02_2024, 15:09:14
./Mamba Training Results/mamba_training_results_p_bird04_01_2024, 12:17:18
./Mamba Training Results/mamba_training_results_p_bird04_01_2024, 14:10:54
./Mamba Training Results/mamba_training_results_p_static-object04_01_2024, 18:40:54
./Mamba Training Results/mamba_training_results_p_static-object04_02_2024, 12:16:07


In [29]:
best_models

,,datetime,Best Result @5,Best Result @10,Best Result @20,Learning Rate,Batch Size,Layers,d_state,Model Path,Test Files,Mamba Config,Training Config,Train Loss History,Evaluation Loss History,Eval History,Best Results,Object
Object,,,,,,,,,,,,,,,,,,
airplane,1,"04/01/2024, 17:45:15",1729822,1329228,1506531,0.010,16,1,16,./Models/mamba_model_airplane_dm8nl1ds16dc4rs4...,[VIS_airplane_20231025-19054_20231025-062708.j...,"{'d_model': 8, 'n_layer': 1, 'd_state': 16, 'e...","{'batch_size': 16, 'optimizer_type': <class 't...","[3691976.34375, 3125740.60546875, 2669021.0771...","[4570303.519042969, 4172886.142403739, 3805460...","{5: [tensor(5452897.5000), tensor(4976200.), t...","{5: tensor(1729822.3750), 10: tensor(1329228.2...",airplane
bird,0,"04/01/2024, 11:38:50",42458,274690,210503,0.010,16,2,16,./Models/mamba_model_bird_dm8nl2ds16dc4rs42bs1...,"[VIS_bird_20231114-735_20231114-082323.json, V...","{'d_model': 8, 'n_layer': 2, 'd_state': 16, 'e...","{'batch_size': 16, 'optimizer_type': <class 't...","[307955.63125, 252278.05223214286, 208267.4618...","[255052.65277777778, 223850.57465277778, 19989...","{5: [tensor(256503.7031), tensor(224283.8281),...","{5: tensor(42458.8398), 10: tensor(274690.1875...",bird
static-object,0,"04/01/2024, 18:40:54",1658464,1724687,2196906,0.010,16,2,16,./Models/mamba_model_static-object_dm8nl2ds16d...,[NIR_static-object_20231024-35607_20231024-190...,"{'d_model': 8, 'n_layer': 2, 'd_state': 16, 'e...","{'batch_size': 16, 'optimizer_type': <class 't...","[12848978.665492957, 11020422.658450704, 93890...","[9451335.125, 8555712.34765625, 7105615.59375,...","{5: [tensor(9157741.), tensor(8292829.5000), t...","{5: tensor(1658464.6250), 10: tensor(1724687.3...",static-object
uav,1,"04/01/2024, 08:38:45",1897,1233,1493,0.005,16,2,16,./Models/mamba_model_uav_dm8nl2ds16dc4rs42bs16...,"[VIS_uav_20231117-18018_20231117-094544.json, ...","{'d_model': 8, 'n_layer': 2, 'd_state': 16, 'e...","{'batch_size': 16, 'optimizer_type': <class 't...","[3892871.183189655, 2568731.167564655, 1634962...","[2925966.65625, 1907339.246875, 1211713.09375,...","{5: [tensor(2807275.5000), tensor(1822063.7500...","{5: tensor(1897.7583), 10: tensor(1233.9237), ...",uav


In [30]:
saved_models_path

{'airplane': './Models/mamba_model_airplane_dm8nl1ds16dc4rs42bs16opAdamlr0.01r104_01_2024, 17:45:15',
 'bird': './Models/mamba_model_bird_dm8nl2ds16dc4rs42bs16opAdamlr0.01r104_01_2024, 11:38:50',
 'static-object': './Models/mamba_model_static-object_dm8nl2ds16dc4rs42bs16opAdamlr0.01r104_01_2024, 18:40:54',
 'uav': './Models/mamba_model_uav_dm8nl2ds16dc4rs42bs16opAdamlr0.005r104_01_2024, 08:38:45'}

In [31]:
durations =[5, 10, 15, 20, 25, 30]
evaluation_config = {'batch_size': 32, 'device' : device, 'evaluation_time': 3, 'delta': 0.04, 'evaluation_durations': durations} #can tale delta from sample config
flying_objects = ['airplane', 'uav', 'bird', 'static-object']
# saved_models_path = {'uav':'mamba_model_uav_dm8nl1ds16dc4rs42bs32lr0.1r3', 'bird': 'mamba_model_bird_dm8nl1ds16dc4rs42bs32lr0.1r3'}
evaluation_results = {}

for f_object in flying_objects:
  best_object_row = best_models.loc[best_models['Object'] == f_object]
  model_path = best_object_row['Model Path'].values[0]
  state = torch.load(model_path, map_location=torch.device('cpu'))

  mamba_config = best_object_row['Mamba Config'].values[0]
  # mamba_config = {'d_model' : 8, 'n_layer': 1, 'd_state' : 16, 'expand' : 2, 'd_conv' : 4, 'model_random_state' : 42}
  args = ModelArgs(mamba_config['d_model'], mamba_config['n_layer'], mamba_config['d_state'], mamba_config['expand'], mamba_config['d_conv'])
  mamba_model = Mamba(args)

  mamba_model.load_state_dict(state['state_dict'])

  evaluating_mamba = Evaluating(mamba_model, evaluation_config)
  all = []
  for dur in durations:
    all.extend(all_test_batches[dur])

  evaluating_mamba.evaluate(all)
  evaluation_results[f_object] = evaluating_mamba.eval_results

In [20]:
evaluation_results['bird'].keys()

dict_keys([5, 10, 15, 20, 25, 30])

In [ ]:
evaluation_results['bird'][30]

###Classification evaluation

In [32]:
subf_dict = {i:subfolders[i] for i in range(len(subfolders))}
labels_dict = {subfolders[i]:i for i in range(len(subfolders))}

In [33]:
# def classify(evaluation_results, all_test_labels):

all_reports = {d:[] for d in durations}
for dur in durations:
  dur_labels = all_test_labels[dur]
  errors = torch.full((len(subf_dict.keys()), len(dur_labels)), torch.inf)
  for f_object, eval_results in evaluation_results.items():
    if len(eval_results[dur]) > 0:
      errors[labels_dict[f_object]] = torch.tensor(eval_results[dur])
  predicted_class = torch.argmin(errors, dim=0)
  report = classification_report(dur_labels, predicted_class, target_names=evaluation_results.keys())
  all_reports[dur] = report
  print(report)
  # return all_reports

               precision    recall  f1-score   support

     airplane       0.60      0.08      0.15        36
          uav       0.30      1.00      0.46        47
         bird       0.33      0.23      0.27        30
static-object       0.52      0.13      0.21        93

     accuracy                           0.33       206
    macro avg       0.44      0.36      0.27       206
 weighted avg       0.46      0.33      0.26       206

               precision    recall  f1-score   support

     airplane       0.67      0.14      0.23        29
          uav       0.32      1.00      0.48        44
         bird       0.20      0.21      0.21        14
static-object       0.61      0.12      0.20        91

     accuracy                           0.35       178
    macro avg       0.45      0.37      0.28       178
 weighted avg       0.52      0.35      0.28       178

               precision    recall  f1-score   support

     airplane       0.75      0.12      0.21        24
   

### run samples on models and produce errors

###evaluate vs labels

In [ ]:
all_test_batches
all_test_labels

In [ ]:
training_config = {'batch_size': 32, 'device' : device, 'evaluation_time': 3, 'delta': 0.04} #can tale delta from sample config

In [ ]:
evaluating_mamba = Evaluating(mamba_model, training_config)
durations = [30]
for dur in durations:
  print(dur)
  evaluating_mamba.evaluate(all_test_batches[dur])
  print(evaluating_mamba.eval_results[dur])


30
[tensor(12489.0312), tensor(5405.4116), tensor(4451.2964), tensor(3382.5276), tensor(5864.8491), tensor(4041.7700), tensor(2746.7488), tensor(205309.7344), tensor(57807.7188), tensor(6495.0254), tensor(18579.4746), tensor(7843.4873), tensor(3412.9260), tensor(1908.4050), tensor(1528.3823), tensor(6983.9355), tensor(18166.3398), tensor(1080.9435), tensor(4024.5134), tensor(43540.8516), tensor(43872.9844), tensor(1854.9661), tensor(19742.0312), tensor(19536.2383), tensor(274.4008), tensor(4323.7715), tensor(1676.1841), tensor(18359.2637), tensor(16552.3008), tensor(149.4890), tensor(312.8918), tensor(1796.8771), tensor(18294.4043), tensor(14484.8721), tensor(15052.1846), tensor(8180.6768), tensor(767.8193), tensor(1591.0153), tensor(878.1680), tensor(19361.5488), tensor(61723.0938), tensor(75288.6797), tensor(53168.8125), tensor(30513.0996), tensor(19446.8730), tensor(47785.7617), tensor(102638.3281), tensor(123514.3438), tensor(64187.2266), tensor(22625.7148), tensor(8850.4521), tens

In [ ]:
print(len(evaluating_mamba.eval_results[dur]))

193


In [ ]:
len(all_test_labels[30])

193

In [ ]:
uav_errors = evaluating_mamba.eval_30sec

In [ ]:
model_path = './Models/' + saved_models_path['bird']
state = torch.load(model_path, map_location=torch.device('cpu'))

mamba_config = {'d_model' : 8, 'n_layer': 1, 'd_state' : 16, 'expand' : 2, 'd_conv' : 4, 'model_random_state' : 42}
args = ModelArgs(mamba_config['d_model'], mamba_config['n_layer'], mamba_config['d_state'], mamba_config['expand'], mamba_config['d_conv'])
mamba_model = Mamba(args)

mamba_model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [ ]:
evaluating_mamba = Evaluating(mamba_model, training_config)
durations = [30]
for dur in durations:
  print(dur)
  evaluating_mamba.evaluate(all_test_batches[dur])
  print(evaluating_mamba.eval_60sec)
  print(evaluating_mamba.eval_30sec)
  print(evaluating_mamba.eval_10sec)
  print(evaluating_mamba.eval_5sec)

30
[]
[tensor(128.3378), tensor(1728.0419), tensor(1439.9631), tensor(117.7386), tensor(3811.1191), tensor(2135.4888), tensor(4431.0791), tensor(20955.5664), tensor(4438.1777), tensor(1405.4056), tensor(4827.9468), tensor(1224.5350), tensor(2012.8898), tensor(1561.7657), tensor(170.7469), tensor(968.8383), tensor(1809.0675), tensor(1072.1622), tensor(201.9536), tensor(487.2441), tensor(514.5453), tensor(403.6157), tensor(2447.4348), tensor(168.1511), tensor(266.2188), tensor(5.4213), tensor(7.1437), tensor(1071.8925), tensor(55191.0586), tensor(49063.4531), tensor(42242.8750), tensor(122.7196), tensor(706.9525), tensor(823.6423), tensor(913.5855), tensor(268.4267), tensor(247.9959), tensor(40.0456), tensor(130.0172), tensor(1513.5565), tensor(471.9264), tensor(4662.0088), tensor(3597.8101), tensor(2905.7825), tensor(2380.9297), tensor(2950.6433), tensor(4955.8218), tensor(7138.9258), tensor(5402.0078), tensor(3160.8430), tensor(3197.2913), tensor(10700.0088), tensor(14966.4434), tensor

In [ ]:
bird_errors = evaluating_mamba.eval_30sec

In [ ]:
for b,u in zip(bird_errors, uav_errors):
  print(b<u)

tensor(True)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(True)
tensor(True)
tensor(False)
tensor(False)
tensor(True)
tensor(False)
tensor(True)
tensor(True)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(True)
tensor(False)
tensor(False)
tensor(True)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(True)
tensor(True)
tensor(False)
tensor(False)
te